In [4]:
import gin, os
import IPython.display as ipd
import torch

from vqvae_timbre.modules import VQTimbreModule
from vqvae_timbre.modules import gin_register_and_parse

%load_ext autoreload
%autoreload 2

gin.enter_interactive_mode()
ginconf = "/homes/cv300/Documents/vq-vae-timbre/gin_configs/vqvae_timbre.gin"
gin_register_and_parse(ginconf)

URMP_DATA_PATH = "/import/c4dm-datasets/URMP-split/np-segments-3/"
MODEL_PATH = 'models/epoch=655-step=149999.ckpt'

PATH = os.path.join(os.pardir, MODEL_PATH)
model = VQTimbreModule.load_from_checkpoint(PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
STFT kernels created, time used = 0.2103 seconds


In [5]:
from vqvae_timbre.datasets.loaders import URMP
from torch.utils.data import DataLoader

ds = URMP(URMP_DATA_PATH, split='test')
test_loader = DataLoader(ds, batch_size=1, shuffle=True)
audio  = iter(test_loader).next()
ipd.Audio(audio.squeeze(1).numpy(), rate=ds.sr)

In [7]:
audio_pred, _, _, _, _ = model(audio)
ipd.Audio(audio_pred.squeeze(0).detach().numpy(), rate=ds.sr)

In [9]:
embedding = model.vq.embedding
decoder = model.decoder
synth = model.synth

In [13]:
n_frames = 10
n_codes = embedding.weight.shape[0]

vq_samples = torch.stack([embedding.weight for i in range(n_frames)]).permute(1, 0, 2)

gain = torch.ones(n_codes, n_frames, model.decoder._p['n_coefficients'])

stride = model.synth._p['stride']
wsize = model.synth._p['wsize']

coeffs = decoder(vq_samples)
audio_out = synth(torch.zeros(n_codes, 1, (n_frames - 1) * stride), gain * coeffs)

In [14]:
from vqvae_timbre.utils import FeatureExtractor
import numpy as np

audio_np = audio_out.detach().numpy().astype(np.float32)

ft = FeatureExtractor(fs=ds.sr)

features = [ft.compute_spectral_features(x) for x in audio_np]

KeyboardInterrupt: 